<a href="https://colab.research.google.com/github/thakkar-hiren/News-Summarisation/blob/main/8_News_Summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Text Summarization Using Doc2Vec Embeddings and TextRank Algorithm:
This file contains an implementation of text summarization using Doc2Vec embeddings and the TextRank algorithm. The process begins by loading a dataset containing news articles and preprocessed text. The necessary libraries, including nltk, numpy, pandas, and gensim, are imported for data manipulation and modeling. The preprocessed text is tokenized into sentences using NLTK's sent_tokenize function, and each sentence is further tokenized into words using word_tokenize. The Doc2Vec model is trained on the tokenized corpus using Gensim's Doc2Vec class, with parameters such as vector size, window size, and minimum word count specified. The sentence embeddings are obtained by inferring vectors for each sentence using the trained Doc2Vec model. Cosine similarity is then used to construct a similarity matrix based on the sentence embeddings. The TextRank algorithm is applied to rank sentences based on their similarity scores, and the top-ranked sentences are selected to form the summary. The number of sentences in the summary is customizable, with options for generating summaries of 3 or 5 sentences. Additionally, the file includes evaluation functions to compute the average ROUGE (Recall-Oriented Understudy for Gisting Evaluation) scores, providing insights into the quality of the generated summaries. This implementation leverages Doc2Vec embeddings to capture semantic information and generate informative summaries, making it suitable for various text summarization tasks across different domains.

# Mounting Drive

In [ ]:
# Mounting Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Importing Important Libraries

In [ ]:
# Importing Important Libraries
import nltk
import numpy as np
import pandas as pd
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize, sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Loading Dataset

In [ ]:
# Loading Dataset
news_data = pd.read_csv('/content/drive/MyDrive/Datasets/filtered_news_data.csv')
news_data

,Unnamed: 0,text,ctext,text_len,ctext_len,text_sent_count,text_word_count,ctext_sent_count,ctext_word_count,preprocessed_text,preprocessed_ctext
0,0,The Administration of Union Territory Daman an...,The Daman and Diu administration on Wednesday ...,358,2313,2,62,16,413,the administration of union territory daman an...,the daman and diu administration on wednesday ...
1,2,The Indira Gandhi Institute of Medical Science...,The Indira Gandhi Institute of Medical Science...,398,2112,3,70,18,379,the indira gandhi institute of medical science...,the indira gandhi institute of medical science...
2,4,Hotels in Maharashtra will train their staff t...,Hotels in Mumbai and other Indian cities are t...,366,3249,3,67,8,569,hotels in maharashtra will train their staff t...,hotels in mumbai and other indian cities are t...
3,5,A 32-year-old man on Wednesday was found hangi...,An alleged suspect in a kidnapping case was fo...,347,2247,3,63,23,440,a 32yearold man on wednesday was found hanging...,an alleged suspect in a kidnapping case was fo...
4,6,The Delhi High Court reduced the compensation ...,"In an interesting ruling, the Delhi high court...",361,2367,3,70,11,444,the delhi high court reduced the compensation ...,in an interesting ruling the delhi high court ...
...,...,...,...,...,...,...,...,...,...,...,...
3911,4507,After emerging as the single largest party in ...,As uncertainty loomed large in the BMC polls d...,335,606,3,66,1,115,after emerging as the single largest party in ...,as uncertainty loomed large in the bmc polls d...
3912,4509,Fruit juice concentrate maker Rasna is eyeing ...,"Mumbai, Feb 23 (PTI) Fruit juice concentrate m...",361,1000,3,70,8,200,fruit juice concentrate maker rasna is eyeing ...,mumbai feb 23 pti fruit juice concentrate make...
3913,4510,Former Indian cricketer Sachin Tendulkar atten...,Former cricketer Sachin Tendulkar was spotted ...,362,933,2,63,3,158,former indian cricketer sachin tendulkar atten...,former cricketer sachin tendulkar was spotted ...
3914,4512,The Maharashtra government has initiated an in...,Maharahstra Power Minister Chandrashekhar Bawa...,387,1397,3,63,9,233,the maharashtra government has initiated an in...,maharahstra power minister chandrashekhar bawa...


# Building Model

In [ ]:
# Tokenize function
def tokenize_text(text):
    return [word_tokenize(sent) for sent in sent_tokenize(text)]

In [ ]:
# Train Doc2Vec model
tokenized_corpus = [word_tokenize(article) for article in news_data['preprocessed_ctext']]
documents = [TaggedDocument(words=words, tags=[str(i)]) for i, words in enumerate(tokenized_corpus)]
doc2vec_model = Doc2Vec(documents, vector_size=100, window=5, min_count=1, workers=4, epochs=20)

In [ ]:
# Sentence embedding function using Doc2Vec
def sentence_embedding(sentence):
    return doc2vec_model.infer_vector(sentence)

In [ ]:
# Calculate similarity matrix based on sentence embeddings
def cosine_similarity_matrix(embeddings):
    similarity_matrix = cosine_similarity(embeddings, embeddings)
    return similarity_matrix

In [ ]:
# TextRank function
def textrank(similarity_matrix):
    damping_factor = 0.85
    n_iterations = 300
    n_sentences = similarity_matrix.shape[0]
    ranks = np.ones(n_sentences) / n_sentences  # Initialize page ranks

    for _ in range(n_iterations):
        new_ranks = (1 - damping_factor) / n_sentences + damping_factor * similarity_matrix.T.dot(ranks)
        ranks = new_ranks
    return ranks

In [ ]:
# Summarization function
def summarize_article(article_text, nos=3):
    tokenized_sentences = tokenize_text(article_text)
    sentence_embeddings = np.array([sentence_embedding(sentence) for sentence in tokenized_sentences])
    similarity_matrix = cosine_similarity_matrix(sentence_embeddings)
    ranks = textrank(similarity_matrix)

    n_summary_sentences = min(nos, len(tokenized_sentences))
    top_sentence_indices = ranks.argsort()[-n_summary_sentences:][::-1]
    predicted_summary = " ".join([" ".join(tokenized_sentences[i]) for i in sorted(top_sentence_indices)])

    return predicted_summary

In [ ]:
# Apply summarization to each article
news_data['predictedSummary_5'] = news_data['preprocessed_ctext'].apply(lambda x: summarize_article(x,nos=5))

In [ ]:
# Apply summarization to each article
news_data['predictedSummary_3'] = news_data['preprocessed_ctext'].apply(lambda x: summarize_article(x))

# Model Evaluation

In [ ]:
!pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=69c0572208d010cef60e58d37186061866a57542d997cb8a7216d1e9a97d02d2
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score


In [ ]:
# Model evaluation function
from rouge_score import rouge_scorer

def evaluate_summaries_3(news_data):
    scorer = rouge_scorer.RougeScorer(rouge_types=['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = []

    for idx, row in news_data.iterrows():
        scores = scorer.score(target=row['preprocessed_text'], prediction=row['predictedSummary_3'])
        rouge_scores.append(scores)

    avg_rouge1_precision = np.mean([score['rouge1'].precision for score in rouge_scores])
    avg_rouge1_recall = np.mean([score['rouge1'].recall for score in rouge_scores])
    avg_rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])

    avg_rouge2_precision = np.mean([score['rouge2'].precision for score in rouge_scores])
    avg_rouge2_recall = np.mean([score['rouge2'].recall for score in rouge_scores])
    avg_rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])

    avg_rougeL_precision = np.mean([score['rougeL'].precision for score in rouge_scores])
    avg_rougeL_recall = np.mean([score['rougeL'].recall for score in rouge_scores])
    avg_rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

    print("Average ROUGE-1 Precision: ", avg_rouge1_precision)
    print("Average ROUGE-1 Recall: ", avg_rouge1_recall)
    print("Average ROUGE-1 F1-Score: ", avg_rouge1_f1)
    print("Average ROUGE-2 Precision: ", avg_rouge2_precision)
    print("Average ROUGE-2 Recall: ", avg_rouge2_recall)
    print("Average ROUGE-2 F1-Score: ", avg_rouge2_f1)
    print("Average ROUGE-L Precision: ", avg_rougeL_precision)
    print("Average ROUGE-L Recall: ", avg_rougeL_recall)
    print("Average ROUGE-L F1-Score: ", avg_rougeL_f1)

In [ ]:
def evaluate_summaries_5(news_data):
    scorer = rouge_scorer.RougeScorer(rouge_types=['rouge1', 'rouge2', 'rougeL'])
    rouge_scores = []

    for idx, row in news_data.iterrows():
        scores = scorer.score(target=row['preprocessed_text'], prediction=row['predictedSummary_5'])
        rouge_scores.append(scores)

    avg_rouge1_precision = np.mean([score['rouge1'].precision for score in rouge_scores])
    avg_rouge1_recall = np.mean([score['rouge1'].recall for score in rouge_scores])
    avg_rouge1_f1 = np.mean([score['rouge1'].fmeasure for score in rouge_scores])

    avg_rouge2_precision = np.mean([score['rouge2'].precision for score in rouge_scores])
    avg_rouge2_recall = np.mean([score['rouge2'].recall for score in rouge_scores])
    avg_rouge2_f1 = np.mean([score['rouge2'].fmeasure for score in rouge_scores])

    avg_rougeL_precision = np.mean([score['rougeL'].precision for score in rouge_scores])
    avg_rougeL_recall = np.mean([score['rougeL'].recall for score in rouge_scores])
    avg_rougeL_f1 = np.mean([score['rougeL'].fmeasure for score in rouge_scores])

    print("Average ROUGE-1 Precision: ", avg_rouge1_precision)
    print("Average ROUGE-1 Recall: ", avg_rouge1_recall)
    print("Average ROUGE-1 F1-Score: ", avg_rouge1_f1)
    print("Average ROUGE-2 Precision: ", avg_rouge2_precision)
    print("Average ROUGE-2 Recall: ", avg_rouge2_recall)
    print("Average ROUGE-2 F1-Score: ", avg_rouge2_f1)
    print("Average ROUGE-L Precision: ", avg_rougeL_precision)
    print("Average ROUGE-L Recall: ", avg_rougeL_recall)
    print("Average ROUGE-L F1-Score: ", avg_rougeL_f1)

In [ ]:
print("For Doc2Vec:- ")
print("Evaluation for the summary of 3 sentences: \n")
evaluate_summaries_3(news_data)
print("\n")
print("Evaluation for the summary of 5 sentences: \n")
evaluate_summaries_5(news_data)

For Doc2Vec:- 
Evaluation for the summary of 3 sentences: 

Average ROUGE-1 Precision:  0.36301952018342026
Average ROUGE-1 Recall:  0.3737610861223355
Average ROUGE-1 F1-Score:  0.35651403825501443
Average ROUGE-2 Precision:  0.1514123008937542
Average ROUGE-2 Recall:  0.15872511292230726
Average ROUGE-2 F1-Score:  0.14985601956868558
Average ROUGE-L Precision:  0.24980213512034669
Average ROUGE-L Recall:  0.2554896980395002
Average ROUGE-L F1-Score:  0.24409437495208847


Evaluation for the summary of 5 sentences: 

Average ROUGE-1 Precision:  0.3156547912176222
Average ROUGE-1 Recall:  0.5270624534750465
Average ROUGE-1 F1-Score:  0.38325331576653004
Average ROUGE-2 Precision:  0.14606143350903011
Average ROUGE-2 Recall:  0.24793282043462483
Average ROUGE-2 F1-Score:  0.1780856213611698
Average ROUGE-L Precision:  0.21524044970023
Average ROUGE-L Recall:  0.3585305572057826
Average ROUGE-L F1-Score:  0.26053875012785394
